In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from user_agents import parse as aparse
from dateutil.parser import parse as dparse
from datetime import datetime
from ip2geotools.databases.noncommercial import DbIpCity
import json

pd.set_option('display.max_columns', 85)
pd.set_option('display.max_rows', 85)

In [2]:
# - Exercici 1
# Normalitza, identifica i enumera cada un dels atributs / variables de l'estructura de l'arxiu
# "Web_access_log-akumenius.com" que trobaràs al repositori de GitHub "Data-sources".

In [3]:
df = pd.read_csv('Web_access_log-akumenius.com.txt', sep='delimiter', header=None, engine='python')

In [4]:
df = pd.read_table('Web_access_log-akumenius.com.txt', delim_whitespace=True, 
                   names=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L'))

C:\Users\ab\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [5]:
df.tail(5)

,A,B,C,D,E,F,G,H,I,J,K,L
261868,www.akumenius.com,5.255.253.53,-,-,[02/Mar/2014:03:05:39,+0100],GET / HTTP/1.1,200,7528,-,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...,VLOG=-
261869,www.akumenius.com,74.86.158.107,-,-,[02/Mar/2014:03:09:52,+0100],HEAD / HTTP/1.1,200,-,-,Mozilla/5.0+(compatible; UptimeRobot/2.0; http...,VLOG=-
261870,localhost,127.0.0.1,-,-,[02/Mar/2014:03:10:18,+0100],OPTIONS * HTTP/1.0,200,-,-,Apache (internal dummy connection),VLOG=-
261871,localhost,127.0.0.1,-,-,[02/Mar/2014:03:10:18,+0100],OPTIONS * HTTP/1.0,200,-,-,Apache (internal dummy connection),VLOG=-
261872,localhost,127.0.0.1,-,-,[02/Mar/2014:03:10:18,+0100],OPTIONS * HTTP/1.0,200,-,-,Apache (internal dummy connection),VLOG=-


In [6]:
#Inspeccionem els valors únics de les diferents columnes. La columna 'C' només té guions
df['F'].unique()

array(['+0100]'], dtype=object)

In [7]:
# l'esborrem
df.drop(['C'], axis = 1, inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261873 entries, 0 to 261872
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   A       261873 non-null  object
 1   B       261873 non-null  object
 2   D       261873 non-null  object
 3   E       261873 non-null  object
 4   F       261873 non-null  object
 5   G       261873 non-null  object
 6   H       261873 non-null  object
 7   I       261873 non-null  object
 8   J       261873 non-null  object
 9   K       261873 non-null  object
 10  L       261873 non-null  object
dtypes: object(11)
memory usage: 22.0+ MB


In [9]:
# La columna 'F' és una continuació de la 'E' (time). Les ajuntem,
df['E'] = df['E'] + ' ' + df['F']
df.drop(['F'], axis = 1, inplace = True)

In [10]:
df.tail(5)

,A,B,D,E,G,H,I,J,K,L
261868,www.akumenius.com,5.255.253.53,-,[02/Mar/2014:03:05:39 +0100],GET / HTTP/1.1,200,7528,-,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...,VLOG=-
261869,www.akumenius.com,74.86.158.107,-,[02/Mar/2014:03:09:52 +0100],HEAD / HTTP/1.1,200,-,-,Mozilla/5.0+(compatible; UptimeRobot/2.0; http...,VLOG=-
261870,localhost,127.0.0.1,-,[02/Mar/2014:03:10:18 +0100],OPTIONS * HTTP/1.0,200,-,-,Apache (internal dummy connection),VLOG=-
261871,localhost,127.0.0.1,-,[02/Mar/2014:03:10:18 +0100],OPTIONS * HTTP/1.0,200,-,-,Apache (internal dummy connection),VLOG=-
261872,localhost,127.0.0.1,-,[02/Mar/2014:03:10:18 +0100],OPTIONS * HTTP/1.0,200,-,-,Apache (internal dummy connection),VLOG=-


In [11]:
#Trobem que a la columna 'D' només hi ha nulls ('-') o el valor 'clarcat'.
df['D'].unique()

array(['-', 'clarcat'], dtype=object)

In [12]:
#Canviem els noms de les columnes:
df.rename(columns={'A':'server', 'B': 'ip', 'D':'status', 'E': 'time',
                   'G':'request', 'H':'http_code', 'I':'bytes', 'J':'from', 'K':'user_agent', 'L':'appendix'},
                     inplace = True)

In [13]:
# BUSQUEM QUINS VALORS CONTENEN ELS DIFERENTS CAMPS:


#Busquem els valors de la columna 'J', trobem només l'apendix 'VLOG=-' i un valor pertanyent a la columna 'user_agent'
df['appendix'].value_counts()

VLOG=-                                                                261871
Mozilla/5.0 (compatible; AhrefsBot/5.0; +http://ahrefs.com/robot/)         2
Name: appendix, dtype: int64

In [14]:
# Hem de trobar els  registres dels dos valors que no encaixen al camp 'appendix' per pasar-los a 'user_agent':

filt = df['appendix'].str.contains('Mozilla') == True
df[filt] 

,server,ip,status,time,request,http_code,bytes,from,user_agent,appendix
181473,www.akumenius.com,5.10.83.41,-,[26/Feb/2014:23:44:22 +0100],GET /escapadas/escapada-especial-san-valentin-...,"HTTP/1.1""",200,3100,-,Mozilla/5.0 (compatible; AhrefsBot/5.0; +http:...
236496,www.akumenius.com,5.10.83.11,-,[28/Feb/2014:16:07:20 +0100],"GET /escapadas/escapada-\al-salto-de-roldan\""-...","HTTP/1.1""",200,3100,-,Mozilla/5.0 (compatible; AhrefsBot/5.0; +http:...


In [15]:
# Trobem el valor 'HTTP/1.1"' que hauria d'anar a la columna 'request' i comprovem quants valors s'han mogut (només 2):
df['http_code'].value_counts()

200          116366
200          110014
304           19660
404            8484
304            5609
301             598
301             272
206             171
403             156
404             146
206             133
302              70
302              39
403              38
502              25
408              21
502              19
408              16
400              13
400              13
401               3
HTTP/1.1"         2
401               2
500               2
500               1
Name: http_code, dtype: int64

In [16]:
df.loc[236496, 'appendix']

'Mozilla/5.0 (compatible; AhrefsBot/5.0; +http://ahrefs.com/robot/)'

In [17]:
# Arreglem aquests 2 valors:

df.loc[181473, ['request', 'http_code', 'bytes', 'from', 'user_agent','appendix']] = ['GET /escapadas/escapada-especial-san-valentin-\\different\\"-en-playa-de-san-juan-hotel-castilla-alicante-80beffa7d651981e2b7abb7ad6a79508.html HTTP/1.1',
                                                                       '200', '3100', '-', 'Mozilla/5.0 (compatible; AhrefsBot/5.0; +http://ahrefs.com/robot/)',
                                                                         'VLOG=-']

In [18]:
df.loc[236496, ['request', 'http_code', 'bytes',
                'from', 'user_agent', 'appendix']] = ['GET /escapadas/escapada-\\al-salto-de-roldan\\"-observatorio-avifauna-hotel-pedro-i-de-aragon-e72dad4b21bdfb8f7285762400e8ba4b.html HTTP/1.1',
                                                     '200', '3100', '-', 'Mozilla/5.0 (compatible; AhrefsBot/5.0; +http://ahrefs.com/robot/)',
                                                      'VLOG=-']

In [19]:
# Comprovem:
df['appendix'].value_counts()

VLOG=-    261873
Name: appendix, dtype: int64

In [20]:
# - Exercici 2
# Neteja, preprocesa, estructura i transforma (dataframe) les dades del registre d'Accés a la web.

In [21]:
df

,server,ip,status,time,request,http_code,bytes,from,user_agent,appendix
0,localhost,127.0.0.1,-,[23/Feb/2014:03:10:31 +0100],OPTIONS * HTTP/1.0,200,-,-,Apache (internal dummy connection),VLOG=-
1,localhost,127.0.0.1,-,[23/Feb/2014:03:10:31 +0100],OPTIONS * HTTP/1.0,200,-,-,Apache (internal dummy connection),VLOG=-
2,localhost,127.0.0.1,-,[23/Feb/2014:03:10:31 +0100],OPTIONS * HTTP/1.0,200,-,-,Apache (internal dummy connection),VLOG=-
3,localhost,127.0.0.1,-,[23/Feb/2014:03:10:31 +0100],OPTIONS * HTTP/1.0,200,-,-,Apache (internal dummy connection),VLOG=-
4,localhost,127.0.0.1,-,[23/Feb/2014:03:10:31 +0100],OPTIONS * HTTP/1.0,200,-,-,Apache (internal dummy connection),VLOG=-
...,...,...,...,...,...,...,...,...,...,...
261868,www.akumenius.com,5.255.253.53,-,[02/Mar/2014:03:05:39 +0100],GET / HTTP/1.1,200,7528,-,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...,VLOG=-
261869,www.akumenius.com,74.86.158.107,-,[02/Mar/2014:03:09:52 +0100],HEAD / HTTP/1.1,200,-,-,Mozilla/5.0+(compatible; UptimeRobot/2.0; http...,VLOG=-
261870,localhost,127.0.0.1,-,[02/Mar/2014:03:10:18 +0100],OPTIONS * HTTP/1.0,200,-,-,Apache (internal dummy connection),VLOG=-
261871,localhost,127.0.0.1,-,[02/Mar/2014:03:10:18 +0100],OPTIONS * HTTP/1.0,200,-,-,Apache (internal dummy connection),VLOG=-


In [22]:
# Què ens diu la columna server?
df['server'].value_counts()

www.akumenius.com     232300
test.akumenius.com     14610
localhost              14127
akumenius.com            742
akumenius.es              94
Name: server, dtype: int64

In [23]:
# Logs amb el verb GET (87'6%)
request_get = df['request'].str.contains('GET')
request_get.value_counts(True)


True     0.87631
False    0.12369
Name: request, dtype: float64

In [24]:
# Logs amb el verb POST (6,3%)
request_post = df['request'].str.contains('POST')
request_post.value_counts(True)

False    0.936313
True     0.063687
Name: request, dtype: float64

In [25]:
# Els valors del'http user authetification:
df['status'].unique()

array(['-', 'clarcat'], dtype=object)

In [26]:
# Veiem les direccions més freqüentades:
df['from'].value_counts()

-                                                                                                                                                                                                                                                                                                                                                                           99547
http://www.akumenius.com/                                                                                                                                                                                                                                                                                                                                                   41394
http://www.akumenius.com/hotel-list                                                                                                                                                                                                                                 

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261873 entries, 0 to 261872
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   server      261873 non-null  object
 1   ip          261873 non-null  object
 2   status      261873 non-null  object
 3   time        261873 non-null  object
 4   request     261873 non-null  object
 5   http_code   261873 non-null  object
 6   bytes       261873 non-null  object
 7   from        261873 non-null  object
 8   user_agent  261873 non-null  object
 9   appendix    261873 non-null  object
dtypes: object(10)
memory usage: 20.0+ MB


In [28]:
# - A quines columnes volem canviar el dtype?
# - User_agent shit

In [29]:
#Canviem els '-' per nulls a tot el dataframe:
df.replace('-', np.nan, inplace=True)

In [30]:
df['status'].unique()

array([nan, 'clarcat'], dtype=object)

In [31]:
df.shape

(261873, 10)

In [32]:
# Quins valors son nulls:
df.isna()

,server,ip,status,time,request,http_code,bytes,from,user_agent,appendix
0,False,False,True,False,False,False,True,True,False,False
1,False,False,True,False,False,False,True,True,False,False
2,False,False,True,False,False,False,True,True,False,False
3,False,False,True,False,False,False,True,True,False,False
4,False,False,True,False,False,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...
261868,False,False,True,False,False,False,False,True,False,False
261869,False,False,True,False,False,False,True,True,False,False
261870,False,False,True,False,False,False,True,True,False,False
261871,False,False,True,False,False,False,True,True,False,False


In [33]:
# Esborrem tots els registres que siguin tot valors nuls:
df.dropna(axis = 'index', how = 'all', inplace = True)

In [34]:
# No canvia res
df.shape

(261873, 10)

In [35]:
df.dtypes

server        object
ip            object
status        object
time          object
request       object
http_code     object
bytes         object
from          object
user_agent    object
appendix      object
dtype: object

In [36]:
# Intentem passar la columna bytes a float
df['bytes'] = df['bytes'].astype(float)

In [37]:
# Ara ja podem sumar el nombre total de bytes transferits:
df['bytes'].sum()

2596632435.0

In [38]:
# Canviem la columna 'time' a timestamp:
df['time']

0         [23/Feb/2014:03:10:31 +0100]
1         [23/Feb/2014:03:10:31 +0100]
2         [23/Feb/2014:03:10:31 +0100]
3         [23/Feb/2014:03:10:31 +0100]
4         [23/Feb/2014:03:10:31 +0100]
                      ...             
261868    [02/Mar/2014:03:05:39 +0100]
261869    [02/Mar/2014:03:09:52 +0100]
261870    [02/Mar/2014:03:10:18 +0100]
261871    [02/Mar/2014:03:10:18 +0100]
261872    [02/Mar/2014:03:10:18 +0100]
Name: time, Length: 261873, dtype: object

In [39]:
# Eliminem els brakets de la columna time per poder parsejar-la-la:
df['time'] = df['time'].str.replace('[', '')
df['time'] = df['time'].str.replace(']', '')

In [40]:
#Creem una funció fent servir datetime.strptime((x), "%d/%b/%Y:%H:%M:%S %z") i fem un apply de la funció a tota la columna:

def update_time(x):
    return datetime.strptime(x, "%d/%b/%Y:%H:%M:%S %z")

df['time'] = df['time'].apply(update_time)

In [41]:
# Comprovem:

df.loc[0, 'time']

Timestamp('2014-02-23 03:10:31+0100', tz='UTC+01:00')

In [42]:
df.dtypes

server                           object
ip                               object
status                           object
time          datetime64[ns, UTC+01:00]
request                          object
http_code                        object
bytes                           float64
from                             object
user_agent                       object
appendix                         object
dtype: object

In [43]:
# Ara per passar-ho a datetime fem servir datetime.fromtimestamp():
#pd.to_datetime(df.time).dt.tz_localize(None)
def update_date(x):
    return x.to_pydatetime()

df['time'].apply(update_date)

0        2014-02-23 03:10:31+01:00
1        2014-02-23 03:10:31+01:00
2        2014-02-23 03:10:31+01:00
3        2014-02-23 03:10:31+01:00
4        2014-02-23 03:10:31+01:00
                    ...           
261868   2014-03-02 03:05:39+01:00
261869   2014-03-02 03:09:52+01:00
261870   2014-03-02 03:10:18+01:00
261871   2014-03-02 03:10:18+01:00
261872   2014-03-02 03:10:18+01:00
Name: time, Length: 261873, dtype: datetime64[ns, UTC+01:00]

In [44]:
df.dtypes

server                           object
ip                               object
status                           object
time          datetime64[ns, UTC+01:00]
request                          object
http_code                        object
bytes                           float64
from                             object
user_agent                       object
appendix                         object
dtype: object

In [45]:
# Finalment, provem la llibreria user_agent per descomposar la diferent informació que rebem en el log de la columna 'user_agent'.
# Agafem un exemple i fent servir un modul epecialitzat per la tasca, n'obtenim el dispositiu des d'on s'ha fet la consulta:

exemple = df['user_agent'].loc[0]
user_agent = aparse(exemple)
user_agent.browser.family

'Other'

In [46]:
# Provem algunes funcionalitats d'aquest mòdul.El navegador:
user_agent.browser

Browser(family='Other', version=(), version_string='')

In [47]:
str(user_agent)

'Other / Other / Other'

In [48]:
# I el sistema operatiu:
user_agent.os

OperatingSystem(family='Other', version=(), version_string='')

In [49]:
example = df.loc[0, 'user_agent']
user_agent = aparse(example)
x = user_agent.browser.family
x

'Other'

In [50]:
# Volem afegir una columna per cada informació rellevant que ens dona la colum a user_agent:
# 1. navegador, 2. sistema operatiu i 3. dispositiu. Comencem per navegador, creant una llista buida:

# ua_browser = []

# for i in df.index:
#     example = df.loc[i, 'user_agent']
#     user_agent = aparse(example)
#     x = user_agent.browser.family
#     ua_browser.append(x)
    

TypeError: expected string or bytes-like object

In [51]:
# Ens surt l'error “Expected string or bytes-like object”. Mirem els dtypes del dataframe. 'user_agent' és object
# que pot ser barreja de tipus de dades o string. 
df.dtypes

server                           object
ip                               object
status                           object
time          datetime64[ns, UTC+01:00]
request                          object
http_code                        object
bytes                           float64
from                             object
user_agent                       object
appendix                         object
dtype: object

In [52]:
# Per si de cas, passem tota la columna a string
df['user_agent'] = df['user_agent'].astype(str)

In [53]:
# Ara ho tornem a provar:

ua_browser = []

for i in df.index:
    example = df.loc[i, 'user_agent']
    user_agent = aparse(example)
    x = user_agent.browser.family
    ua_browser.append(x)
    

In [110]:
ua_browser

['Other',
 'Other',
 'Other',
 'Other',
 'Other',
 'Other',
 'Other',
 'Other',
 'Other',
 'Other',
 'Other',
 'Other',
 'Other',
 'Other',
 'Other',
 'Other',
 'Other',
 'Other',
 'Other',
 'Other',
 'Other',
 'Other',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'YandexBot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'bingbot',
 'Googlebot',
 'Googlebot',
 'Firefox',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'Googlebot',
 'AhrefsBot',
 'Baiduspider',
 'Googlebot',
 'Googlebot',
 'Baiduspider',
 'Baiduspider',
 'Baiduspider',
 'Baiduspider',
 'Googlebot',
 'Go

In [55]:
# Funciona! ho apliquem a la informació de sistema operatiu i dispositiu:
ua_os = []

for i in df.index:
    example = df.loc[i, 'user_agent']
    user_agent = aparse(example)
    x = user_agent.os.family
    ua_os.append(x)

In [56]:
ua_device = []

for i in df.index:
    example = df.loc[i, 'user_agent']
    user_agent = aparse(example)
    x = user_agent.device.model
    ua_device.append(x)

In [57]:
# Afegim les columnes al df:
df['browser'], df['os'], df['device'] = [ua_browser, ua_os, ua_device]

In [58]:
df.head(5)

,server,ip,status,time,request,http_code,bytes,from,user_agent,appendix,browser,os,device
0,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection),VLOG=-,Other,Other,None
1,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection),VLOG=-,Other,Other,None
2,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection),VLOG=-,Other,Other,None
3,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection),VLOG=-,Other,Other,None
4,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection),VLOG=-,Other,Other,None


In [59]:
df['browser'].unique()

array(['Other', 'Googlebot', 'YandexBot', 'bingbot', 'Firefox',
       'AhrefsBot', 'Baiduspider', 'UptimeRobot', 'msnbot-media',
       'BingPreview', 'Wget', 'ia_archiver', 'spbot', 'Opera', 'IE',
       'Chrome', 'Googlebot-Mobile', 'Mobile Safari', 'Googlebot-Image',
       'MJ12bot', 'Android', 'Google', 'Sogou web spider', 'YandexImages',
       'Chrome Mobile', 'FacebookBot', 'Java', 'msnbot', 'Thunderbird',
       'Baiduspider-image', 'BLEXBot', 'BlackBerry WebKit', 'curl',
       'favicon', 'EasouSpider', 'MobileSafari', 'Chrome Mobile iOS',
       'Mobile Safari UI/WKWebView', 'Safari', 'CareerBot',
       'Firefox Mobile', 'URLAppendBot', 'Exabot', 'socialbm_bot',
       'Chrome Frame', 'Mercury', 'GmailImageProxy', 'Genieo',
       'Chrome Mobile WebView', 'Maxthon', 'Outlook', 'SISTRIX Crawler',
       'Crawler', 'Chromium', 'UC Browser', 'archive.org_bot',
       'Opera Mini', 'AdsBot-Google', 'Xenu Link Sleuth', 'SeznamBot',
       'IE Mobile', 'Twitterbot', 'Yahoo! Slur

In [60]:
df['browser'].value_counts()

Googlebot               51739
Chrome                  39993
Firefox                 34770
IE                      33414
Android                 22012
                        ...  
Mediapartners-Google        1
PaperLiBot                  1
Xenu Link Sleuth            1
Galeon                      1
Yahoo! Slurp                1
Name: browser, Length: 93, dtype: int64

In [61]:
# Passem a la classificació dels diferents http codes. 
df.dtypes

server                           object
ip                               object
status                           object
time          datetime64[ns, UTC+01:00]
request                          object
http_code                        object
bytes                           float64
from                             object
user_agent                       object
appendix                         object
browser                          object
os                               object
device                           object
dtype: object

In [62]:
# Passem la columna 'http_code' primer a str per normalitzar els seus valors:
df['http_code'] = df['http_code'].astype(str)

In [63]:
df['http_code'].unique()

array(['200', '404', '304', '502', '301', '403', '408', '400', '206',
       '401', '302', '500'], dtype=object)

In [64]:
# Veiem que tots els valors estan repetits (200, '200', 304, '304', 301, '301' etc). Ho arreglem:
for i in df['http_code']:
    i.replace("'", "")

In [65]:
# Passem a int per poder operar 
df['http_code'] = df['http_code'].astype(int)

In [66]:
# Mirem els diferents tipus de codis http:
df['http_code'].unique()

array([200, 404, 304, 502, 301, 403, 408, 400, 206, 401, 302, 500])

In [67]:
df['http_code'].value_counts()

200    226382
304     25269
404      8630
301       870
206       304
403       194
302       109
502        44
408        37
400        26
401         5
500         3
Name: http_code, dtype: int64

In [68]:
len(df['http_code'].value_counts())

12

In [106]:
list_errors = ['Success: OK', 'Success: Partial Content (RFC 7233)', 'Redirection: Moved Permanently', 
               'Redirection: Found (Previously "Moved temporarily")', 'Redirection: Not Modified (RFC 7232)',
              'Client Errors: Bad Request', 'Client Errors: Unauthorized (RFC 7235)', 'Client Errors: Forbidden',
              'Client Errors: Not Found', 'Client Errors: Request Timeout', 'Server Errors: Internal Server Error',
              'Server Errors: Bad Gateway']

In [107]:
explicit_http = {}
contador = 0
for i in df['http_code'].unique():
    explicit_http[i] = list_errors[contador]
    contador += 1

In [108]:
explicit_http

{200: 'Success: OK',
 404: 'Success: Partial Content (RFC 7233)',
 304: 'Redirection: Moved Permanently',
 502: 'Redirection: Found (Previously "Moved temporarily")',
 301: 'Redirection: Not Modified (RFC 7232)',
 403: 'Client Errors: Bad Request',
 408: 'Client Errors: Unauthorized (RFC 7235)',
 400: 'Client Errors: Forbidden',
 206: 'Client Errors: Not Found',
 401: 'Client Errors: Request Timeout',
 302: 'Server Errors: Internal Server Error',
 500: 'Server Errors: Bad Gateway'}

In [69]:
# Creem una nova llista buida on afegirem el significat de cada codi a partir d'un loop:
explicit_http = []

for i in df.index:
    x = df.loc[i, 'http_code']
    if x == 200:
        x = 'Success: OK'
        explicit_http.append(x)
    if x == 206:
        x = 'Success: Partial Content (RFC 7233)'
        explicit_http.append(x)
    if x == 301:
        x = 'Redirection: Moved Permanently'
        explicit_http.append(x)
    if x == 302:
        x = 'Redirection: Found (Previously "Moved temporarily")'
        explicit_http.append(x)
    if x == 304:
        x = 'Redirection: Not Modified (RFC 7232)'
        explicit_http.append(x)
    if x == 400:
        x = 'Client Errors: Bad Request'
        explicit_http.append(x)
    if x == 401:
        x = 'Client Errors: Unauthorized (RFC 7235)'
        explicit_http.append(x)
    if x == 403:
        x = 'Client Errors: Forbidden'
        explicit_http.append(x)
    if x == 404:
        x = 'Client Errors: Not Found'
        explicit_http.append(x)
    if x == 408:
        x = 'Client Errors: Request Timeout'
        explicit_http.append(x)
    if x == 500:
        x = 'Server Errors: Internal Server Error'
        explicit_http.append(x)
    if x == 502:
        x = 'Server Errors: Bad Gateway'
        explicit_http.append(x)
    
    else:
        pass
        

In [70]:
explicit_http

['Success: OK',
 'Success: OK',
 'Success: OK',
 'Success: OK',
 'Success: OK',
 'Success: OK',
 'Success: OK',
 'Success: OK',
 'Success: OK',
 'Success: OK',
 'Success: OK',
 'Success: OK',
 'Success: OK',
 'Success: OK',
 'Success: OK',
 'Success: OK',
 'Success: OK',
 'Success: OK',
 'Success: OK',
 'Success: OK',
 'Success: OK',
 'Success: OK',
 'Client Errors: Not Found',
 'Client Errors: Not Found',
 'Client Errors: Not Found',
 'Client Errors: Not Found',
 'Success: OK',
 'Client Errors: Not Found',
 'Client Errors: Not Found',
 'Client Errors: Not Found',
 'Success: OK',
 'Client Errors: Not Found',
 'Client Errors: Not Found',
 'Client Errors: Not Found',
 'Success: OK',
 'Client Errors: Not Found',
 'Client Errors: Not Found',
 'Client Errors: Not Found',
 'Client Errors: Not Found',
 'Success: OK',
 'Client Errors: Not Found',
 'Success: OK',
 'Client Errors: Not Found',
 'Client Errors: Not Found',
 'Client Errors: Not Found',
 'Client Errors: Not Found',
 'Success: OK',
 

In [71]:
# Passem la llista al df
df['explicit_http_code'] = explicit_http

In [72]:
df.head(5)

,server,ip,status,time,request,http_code,bytes,from,user_agent,appendix,browser,os,device,explicit_http_code
0,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection),VLOG=-,Other,Other,None,Success: OK
1,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection),VLOG=-,Other,Other,None,Success: OK
2,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection),VLOG=-,Other,Other,None,Success: OK
3,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection),VLOG=-,Other,Other,None,Success: OK
4,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,NaN,NaN,Apache (internal dummy connection),VLOG=-,Other,Other,None,Success: OK


In [73]:
# Reordanem les columnes per millorar la lògica de lectura:
df = df[['server', 'ip', 'status', 'time', 'request', 'http_code', 'explicit_http_code', 'bytes',
       'from', 'user_agent', 'browser', 'os', 'device','appendix']]

In [74]:
df.head(5)

,server,ip,status,time,request,http_code,explicit_http_code,bytes,from,user_agent,browser,os,device,appendix
0,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,Success: OK,NaN,NaN,Apache (internal dummy connection),Other,Other,None,VLOG=-
1,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,Success: OK,NaN,NaN,Apache (internal dummy connection),Other,Other,None,VLOG=-
2,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,Success: OK,NaN,NaN,Apache (internal dummy connection),Other,Other,None,VLOG=-
3,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,Success: OK,NaN,NaN,Apache (internal dummy connection),Other,Other,None,VLOG=-
4,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,Success: OK,NaN,NaN,Apache (internal dummy connection),Other,Other,None,VLOG=-


In [75]:
# - Exercici 3
# Geolocalitza les IP's. Aqui tens una pagina de interes :
# IP2Locattion: https://blog.ip2location.com/knowledge-base/how-to-add-a-sub-account-in-ip2location/

In [76]:
#Primer mirem les ips uniques

ips = df['ip'].unique()
ips

array(['127.0.0.1', '66.249.76.216', '5.255.253.53', ...,
       '217.174.248.179', '188.135.173.80', '206.198.5.33'], dtype=object)

In [77]:
len(ips)
# Tenim 2921 ips diferents

2921

In [78]:
#Prova de geolocalitzar una ip:

response = DbIpCity.get('66.249.76.216', api_key='free').to_json()
response

'{"ip_address": "66.249.76.216", "city": "Mountain View", "region": "California", "country": "US", "latitude": 37.3893889, "longitude": -122.0832101}'

In [79]:
# Agafarem una mostra de 100 ips del df, ja que fer-les totes és fa molt complicat 
# per temps de processament i límits en l'eina de geolocalització
ips = np.delete(ips, [0])
ips_100 = ips[0:100]

In [80]:
ips_list = []

for ip in ips_100: 
    x = DbIpCity.get(ip, api_key='free').to_json()
    ips_list.append(x)

In [81]:
ips_list

['{"ip_address": "66.249.76.216", "city": "Mountain View", "region": "California", "country": "US", "latitude": 37.3893889, "longitude": -122.0832101}',
 '{"ip_address": "5.255.253.53", "city": "Moscow (Tsentralnyy administrativnyy okrug)", "region": "Moscow", "country": "RU", "latitude": null, "longitude": null}',
 '{"ip_address": "157.55.35.112", "city": "Newark", "region": "New Jersey", "country": "US", "latitude": 40.735657, "longitude": -74.1723667}',
 '{"ip_address": "193.111.141.116", "city": "Amsterdam (Nieuwmarkt en Lastage)", "region": "North Holland", "country": "NL", "latitude": 52.3674313, "longitude": 4.9012206}',
 '{"ip_address": "5.10.83.71", "city": "London", "region": "England", "country": "GB", "latitude": 51.5073219, "longitude": -0.1276474}',
 '{"ip_address": "180.76.5.223", "city": "Beijing", "region": "Beijing", "country": "CN", "latitude": 39.906217, "longitude": 116.3912757}',
 '{"ip_address": "180.76.6.148", "city": "Beijing", "region": "Beijing", "country": "

In [82]:
len(ips_list)

100

In [83]:
import json
ips_json = json.dumps(ips_list)

In [84]:
ips_json

'["{\\"ip_address\\": \\"66.249.76.216\\", \\"city\\": \\"Mountain View\\", \\"region\\": \\"California\\", \\"country\\": \\"US\\", \\"latitude\\": 37.3893889, \\"longitude\\": -122.0832101}", "{\\"ip_address\\": \\"5.255.253.53\\", \\"city\\": \\"Moscow (Tsentralnyy administrativnyy okrug)\\", \\"region\\": \\"Moscow\\", \\"country\\": \\"RU\\", \\"latitude\\": null, \\"longitude\\": null}", "{\\"ip_address\\": \\"157.55.35.112\\", \\"city\\": \\"Newark\\", \\"region\\": \\"New Jersey\\", \\"country\\": \\"US\\", \\"latitude\\": 40.735657, \\"longitude\\": -74.1723667}", "{\\"ip_address\\": \\"193.111.141.116\\", \\"city\\": \\"Amsterdam (Nieuwmarkt en Lastage)\\", \\"region\\": \\"North Holland\\", \\"country\\": \\"NL\\", \\"latitude\\": 52.3674313, \\"longitude\\": 4.9012206}", "{\\"ip_address\\": \\"5.10.83.71\\", \\"city\\": \\"London\\", \\"region\\": \\"England\\", \\"country\\": \\"GB\\", \\"latitude\\": 51.5073219, \\"longitude\\": -0.1276474}", "{\\"ip_address\\": \\"180.76

In [85]:
df_ips= pd.read_json(ips_json)

In [86]:
df_ips.head(5)

,0
0,"{""ip_address"": ""66.249.76.216"", ""city"": ""Mount..."
1,"{""ip_address"": ""5.255.253.53"", ""city"": ""Moscow..."
2,"{""ip_address"": ""157.55.35.112"", ""city"": ""Newar..."
3,"{""ip_address"": ""193.111.141.116"", ""city"": ""Ams..."
4,"{""ip_address"": ""5.10.83.71"", ""city"": ""London"",..."


In [87]:
df_ips.columns = ['A']

In [88]:
df_ips.columns

Index(['A'], dtype='object')

In [89]:
df_ips = pd.DataFrame(df_ips.A.str.split(',',5).tolist(),
                                 columns = ['ip_adress','city', 'region', 'country', 'latitude', 'longitude'])

In [90]:
df_ips.head(5)

,ip_adress,city,region,country,latitude,longitude
0,"{""ip_address"": ""66.249.76.216""","""city"": ""Mountain View""","""region"": ""California""","""country"": ""US""","""latitude"": 37.3893889","""longitude"": -122.0832101}"
1,"{""ip_address"": ""5.255.253.53""","""city"": ""Moscow (Tsentralnyy administrativnyy...","""region"": ""Moscow""","""country"": ""RU""","""latitude"": null","""longitude"": null}"
2,"{""ip_address"": ""157.55.35.112""","""city"": ""Newark""","""region"": ""New Jersey""","""country"": ""US""","""latitude"": 40.735657","""longitude"": -74.1723667}"
3,"{""ip_address"": ""193.111.141.116""","""city"": ""Amsterdam (Nieuwmarkt en Lastage)""","""region"": ""North Holland""","""country"": ""NL""","""latitude"": 52.3674313","""longitude"": 4.9012206}"
4,"{""ip_address"": ""5.10.83.71""","""city"": ""London""","""region"": ""England""","""country"": ""GB""","""latitude"": 51.5073219","""longitude"": -0.1276474}"


In [91]:
# Netegem els registres. Treiem tot el que no necessitem, informació que ja tenim a les columnes i caràcters que sobren:

df_ips=df_ips.replace('\"','',regex=True)
df_ips

,ip_adress,city,region,country,latitude,longitude
0,{ip_address: 66.249.76.216,city: Mountain View,region: California,country: US,latitude: 37.3893889,longitude: -122.0832101}
1,{ip_address: 5.255.253.53,city: Moscow (Tsentralnyy administrativnyy ok...,region: Moscow,country: RU,latitude: null,longitude: null}
2,{ip_address: 157.55.35.112,city: Newark,region: New Jersey,country: US,latitude: 40.735657,longitude: -74.1723667}
3,{ip_address: 193.111.141.116,city: Amsterdam (Nieuwmarkt en Lastage),region: North Holland,country: NL,latitude: 52.3674313,longitude: 4.9012206}
4,{ip_address: 5.10.83.71,city: London,region: England,country: GB,latitude: 51.5073219,longitude: -0.1276474}
...,...,...,...,...,...,...
95,{ip_address: 180.76.6.56,city: Beijing,region: Beijing,country: CN,latitude: 39.906217,longitude: 116.3912757}
96,{ip_address: 180.76.5.152,city: Beijing,region: Beijing,country: CN,latitude: 39.906217,longitude: 116.3912757}
97,{ip_address: 180.76.5.155,city: Beijing,region: Beijing,country: CN,latitude: 39.906217,longitude: 116.3912757}
98,{ip_address: 180.76.6.153,city: Beijing,region: Beijing,country: CN,latitude: 39.906217,longitude: 116.3912757}


In [92]:
df_ips=df_ips.replace('\{','',regex=True)

In [93]:
df_ips=df_ips.replace('\}','',regex=True)

In [94]:
df_ips=df_ips.replace('\:','',regex=True)

In [95]:
df_ips=df_ips.replace('^ip_address','',regex=True)

In [96]:
df_ips=df_ips.replace('city','',regex=True)

In [97]:
df_ips=df_ips.replace('region','',regex=True)

In [98]:
df_ips=df_ips.replace('country','',regex=True)

In [99]:
df_ips=df_ips.replace('latitude','',regex=True)

In [100]:
df_ips=df_ips.replace('longitude','',regex=True)
df_ips

,ip_adress,city,region,country,latitude,longitude
0,66.249.76.216,Mountain View,California,US,37.3893889,-122.0832101
1,5.255.253.53,Moscow (Tsentralnyy administrativnyy okrug),Moscow,RU,null,null
2,157.55.35.112,Newark,New Jersey,US,40.735657,-74.1723667
3,193.111.141.116,Amsterdam (Nieuwmarkt en Lastage),North Holland,NL,52.3674313,4.9012206
4,5.10.83.71,London,England,GB,51.5073219,-0.1276474
...,...,...,...,...,...,...
95,180.76.6.56,Beijing,Beijing,CN,39.906217,116.3912757
96,180.76.5.152,Beijing,Beijing,CN,39.906217,116.3912757
97,180.76.5.155,Beijing,Beijing,CN,39.906217,116.3912757
98,180.76.6.153,Beijing,Beijing,CN,39.906217,116.3912757


In [101]:
df_ips.info

<bound method DataFrame.info of            ip_adress                                           city  \
0      66.249.76.216                                  Mountain View   
1       5.255.253.53    Moscow (Tsentralnyy administrativnyy okrug)   
2      157.55.35.112                                         Newark   
3    193.111.141.116              Amsterdam (Nieuwmarkt en Lastage)   
4         5.10.83.71                                         London   
..               ...                                            ...   
95       180.76.6.56                                        Beijing   
96      180.76.5.152                                        Beijing   
97      180.76.5.155                                        Beijing   
98      180.76.6.153                                        Beijing   
99      180.76.6.151                                        Beijing   

             region country      latitude       longitude  
0        California      US    37.3893889    -122.08321

In [102]:
# - Exercici 4
# Mostreu-me la teva creativitat, Sorprèn-me fes un pas més enllà amb el analysis anterior.

In [ ]:
# FALTA PER MIRAR:
# -datetime (ens pot servir tal qual està) alex
# -geolocalitzacio: fer una taula apart amb les sis columnes i que s'afegeixi cada ip que rebi: (Y)

# -user-agent shit (Y), es pot ampliar a versions, tàctil, etc però ja sabem fer-ho servir i agafem les funcions principals.
#     Potser això va a l'exercici 1? Com ordeno la feina segons els exerecicis? és important?
# -http code: classificar entre 1xx, 2xx, 3xx, 4xx, 5xx (Y)
# - BOTS 

In [ ]:
# Volem saber quins bots son perjudicials per al nostre servidor:

In [116]:
# Passem la llista de navegadors a set per rebren només els unique values i reconvertim el set a llista:
list_set = set(ua_browser)
unique_list = (list(list_set))

In [117]:
unique_list

['Java',
 'EasouSpider',
 'Amazon Silk',
 'URLAppendBot',
 'Safari',
 'GmailImageProxy',
 'Camino',
 'Yahoo! Slurp',
 'Wget',
 'spider',
 'Googlebot-Image',
 'UC Browser',
 'PaperLiBot',
 'Exabot',
 'archive.org_bot',
 'meanpathbot',
 'AhrefsBot',
 'NING',
 'Netscape',
 'IE',
 'Baiduspider-image',
 'KomodiaBot',
 'Mercury',
 'Googlebot',
 'Thunderbird',
 'Sogou web spider',
 'SurveyBot',
 'Chromium',
 'YandexImages',
 'CareerBot',
 'Firefox',
 'Maxthon',
 'Mobile Safari UI/WKWebView',
 'Crawler',
 'favicon',
 'Opera',
 'Googlebot-Mobile',
 'Xenu Link Sleuth',
 'Crazy Browser',
 'YandexBot',
 'IE Mobile',
 'Puffin',
 'MJ12bot',
 'MobileSafari',
 'Python Requests',
 'Firefox Mobile',
 'Outlook',
 'ia_archiver',
 'Python-urllib',
 'SISTRIX Crawler',
 'Apache-HttpClient',
 'curl',
 'oBot',
 'Chrome Frame',
 '360Spider',
 'socialbm_bot',
 'msnbot-media',
 'Chrome Mobile',
 'Android',
 'Opera Mobile',
 'Other',
 'bingbot',
 'Nutch',
 'GrapeshotCrawler',
 'Chrome Mobile WebView',
 'niki-bot',

In [128]:
# Dins els navegadors busquem els bots:
from re import search

sub = '(?i)bot'
llista_bots = []

for i in unique_list:
    if search(sub, i):
        llista_bots.append(i)
    else:
        pass

In [129]:
llista_bots

['URLAppendBot',
 'Googlebot-Image',
 'PaperLiBot',
 'Exabot',
 'archive.org_bot',
 'meanpathbot',
 'AhrefsBot',
 'KomodiaBot',
 'Googlebot',
 'SurveyBot',
 'CareerBot',
 'Googlebot-Mobile',
 'YandexBot',
 'MJ12bot',
 'oBot',
 'socialbm_bot',
 'msnbot-media',
 'bingbot',
 'niki-bot',
 'CompSpyBot',
 'spbot',
 'LoadTimeBot',
 'SnapPreviewBot',
 'msnbot',
 'TweetmemeBot',
 'AdsBot-Google',
 'BLEXBot',
 'SeznamBot',
 'rogerbot',
 'UptimeRobot',
 'FacebookBot',
 'Twitterbot']

In [136]:
df_browser = pd.DataFrame(ua_browser)

In [137]:
df_browser.value_counts()

Googlebot               51739
Chrome                  39993
Firefox                 34770
IE                      33414
Android                 22012
                        ...  
Xenu Link Sleuth            1
Firefox Beta                1
Mediapartners-Google        1
SnapPreviewBot              1
Yahoo! Slurp                1
Length: 93, dtype: int64

In [ ]:
from re import search

sub = '(?i)bot'
llista_bots = []

for i in df_browser:
    if search(sub, i):
        llista_bots.append(i)
    else:
        pass

In [138]:
df

,server,ip,status,time,request,http_code,explicit_http_code,bytes,from,user_agent,browser,os,device,appendix
0,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,Success: OK,NaN,NaN,Apache (internal dummy connection),Other,Other,None,VLOG=-
1,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,Success: OK,NaN,NaN,Apache (internal dummy connection),Other,Other,None,VLOG=-
2,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,Success: OK,NaN,NaN,Apache (internal dummy connection),Other,Other,None,VLOG=-
3,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,Success: OK,NaN,NaN,Apache (internal dummy connection),Other,Other,None,VLOG=-
4,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200,Success: OK,NaN,NaN,Apache (internal dummy connection),Other,Other,None,VLOG=-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261868,www.akumenius.com,5.255.253.53,NaN,2014-03-02 03:05:39+01:00,GET / HTTP/1.1,200,Success: OK,7528.0,NaN,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...,YandexBot,Other,Desktop,VLOG=-
261869,www.akumenius.com,74.86.158.107,NaN,2014-03-02 03:09:52+01:00,HEAD / HTTP/1.1,200,Success: OK,NaN,NaN,Mozilla/5.0+(compatible; UptimeRobot/2.0; http...,UptimeRobot,Other,Desktop,VLOG=-
261870,localhost,127.0.0.1,NaN,2014-03-02 03:10:18+01:00,OPTIONS * HTTP/1.0,200,Success: OK,NaN,NaN,Apache (internal dummy connection),Other,Other,None,VLOG=-
261871,localhost,127.0.0.1,NaN,2014-03-02 03:10:18+01:00,OPTIONS * HTTP/1.0,200,Success: OK,NaN,NaN,Apache (internal dummy connection),Other,Other,None,VLOG=-


In [143]:
is_bot = []
for i in df.index:
    example = df.loc[i, 'user_agent']
    user_agent = aparse(example)
    x = user_agent.is_bot
    is_bot.append(x)

In [149]:
bot = pd.Series(is_bot)

In [152]:
df.loc['is_bot'] = bot

C:\Users\ab\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [153]:
df.columns

Index(['server', 'ip', 'status', 'time', 'request', 'http_code',
       'explicit_http_code', 'bytes', 'from', 'user_agent', 'browser', 'os',
       'device', 'appendix', 'is_bot'],
      dtype='object')

In [154]:
df

,server,ip,status,time,request,http_code,explicit_http_code,bytes,from,user_agent,browser,os,device,appendix,is_bot
0,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200.0,Success: OK,NaN,NaN,Apache (internal dummy connection),Other,Other,None,VLOG=-,0.0
1,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200.0,Success: OK,NaN,NaN,Apache (internal dummy connection),Other,Other,None,VLOG=-,0.0
2,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200.0,Success: OK,NaN,NaN,Apache (internal dummy connection),Other,Other,None,VLOG=-,0.0
3,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200.0,Success: OK,NaN,NaN,Apache (internal dummy connection),Other,Other,None,VLOG=-,0.0
4,localhost,127.0.0.1,NaN,2014-02-23 03:10:31+01:00,OPTIONS * HTTP/1.0,200.0,Success: OK,NaN,NaN,Apache (internal dummy connection),Other,Other,None,VLOG=-,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261869,www.akumenius.com,74.86.158.107,NaN,2014-03-02 03:09:52+01:00,HEAD / HTTP/1.1,200.0,Success: OK,NaN,NaN,Mozilla/5.0+(compatible; UptimeRobot/2.0; http...,UptimeRobot,Other,Desktop,VLOG=-,1.0
261870,localhost,127.0.0.1,NaN,2014-03-02 03:10:18+01:00,OPTIONS * HTTP/1.0,200.0,Success: OK,NaN,NaN,Apache (internal dummy connection),Other,Other,None,VLOG=-,0.0
261871,localhost,127.0.0.1,NaN,2014-03-02 03:10:18+01:00,OPTIONS * HTTP/1.0,200.0,Success: OK,NaN,NaN,Apache (internal dummy connection),Other,Other,None,VLOG=-,0.0
261872,localhost,127.0.0.1,NaN,2014-03-02 03:10:18+01:00,OPTIONS * HTTP/1.0,200.0,Success: OK,NaN,NaN,Apache (internal dummy connection),Other,Other,None,VLOG=-,0.0


In [156]:
df['is_bot'].value_counts()

0.0    178977
1.0     82896
Name: is_bot, dtype: int64

In [162]:
bot_grp = df.groupby(['is_bot'])

In [163]:
df['browser'].unique()

array(['Other', 'Googlebot', 'YandexBot', 'bingbot', 'Firefox',
       'AhrefsBot', 'Baiduspider', 'UptimeRobot', 'msnbot-media',
       'BingPreview', 'Wget', 'ia_archiver', 'spbot', 'Opera', 'IE',
       'Chrome', 'Googlebot-Mobile', 'Mobile Safari', 'Googlebot-Image',
       'MJ12bot', 'Android', 'Google', 'Sogou web spider', 'YandexImages',
       'Chrome Mobile', 'FacebookBot', 'Java', 'msnbot', 'Thunderbird',
       'Baiduspider-image', 'BLEXBot', 'BlackBerry WebKit', 'curl',
       'favicon', 'EasouSpider', 'MobileSafari', 'Chrome Mobile iOS',
       'Mobile Safari UI/WKWebView', 'Safari', 'CareerBot',
       'Firefox Mobile', 'URLAppendBot', 'Exabot', 'socialbm_bot',
       'Chrome Frame', 'Mercury', 'GmailImageProxy', 'Genieo',
       'Chrome Mobile WebView', 'Maxthon', 'Outlook', 'SISTRIX Crawler',
       'Crawler', 'Chromium', 'UC Browser', 'archive.org_bot',
       'Opera Mini', 'AdsBot-Google', 'Xenu Link Sleuth', 'SeznamBot',
       'IE Mobile', 'Twitterbot', 'Yahoo! Slur

In [167]:
bot_grp['browser'].value_counts(normalize = True)

is_bot  browser         
0.0     Chrome              0.219481
        Firefox             0.194271
        IE                  0.186694
        Android             0.122988
        Mobile Safari       0.101762
                              ...   
1.0     LoadTimeBot         0.000012
        PaperLiBot          0.000012
        SnapPreviewBot      0.000012
        Xenu Link Sleuth    0.000012
        Yahoo! Slurp        0.000012
Name: browser, Length: 95, dtype: float64

Per més endavant: mirar de distingir entre bots 'bons' i potencialment perjudicials